<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [5]:
import os
import io
import sys
import re
import csv
import pandas as pd
from tqdm import tqdm_notebook, tnrange
from datetime import datetime

In [12]:
re_From = re.compile(r'^\s?From\:\s?(.*)')
re_Sent = re.compile(r'^Sent\:\s?(.*)')
re_To = re.compile(r'^To\:\s?(.*)')
re_CC = re.compile(r'^C[Cc]\:\s?(.*)')
re_BCC = re.compile(r'^B[Cc][Cc]\:\s?(.*)')
re_Subject = re.compile(r'^Subject\:\s?(.*)')
re_Attachments = re.compile(r'^Attachments\:\s?(.*)')
re_Importance = re.compile(r'Importance\:\s?(.*)')
re_isThread = re.compile(r'isThread\:\s?(.*)')
re_isAutoMessage = re.compile(r'isAutoMessage\:\s?(.*)')
re_isDisplacement = re.compile(r'isDisplacement\:\s?(.*)')
re_hasAllCapLine = re.compile(r'hasAllCapLine\:\s?(.*)')
re_hasBadDate = re.compile(r'hasBadDate\:\s?(.*)')
re_Body = re.compile(r'Body\:\s?')


In [15]:
def get_headers(inputpath): 
    dictionary = {}
    foundBody = False
    filenm = inputpath.split('/')[-1]
    dictionary['filenm'] = From
    
    with open(inputpath, 'r',encoding='utf8') as f:
        From = ''
        Sent = ''
        To = ''
        CC = ''
        Bcc = ''
        Subject = ''
        Attachments = ''
        Importance = ''
        list_body = []
        
        isThread = False
        isAutoMessage = False
        isDisplacement = False
        hasAllCapLine = False
        hasBadDate = False
        
        for line in f.readlines():
            #\f is a special character (dot)
            line = re.sub('\f', '', line)
            line = line.strip()
            if foundBody:
                line = re.sub('[^A-Za-z0-9\s\(\)\/\\\&\?\!\,\.\-\@\<\>\:\+\%\;]', '', line)
                list_body.append(line)
            else:
                if re_From.match(line):
                    From = re_From.findall(line)[0]
                elif re_Sent.match(line):
                    Sent = re_Sent.findall(line)[0]
                elif re_To.match(line):
                    To = re_To.findall(line)[0]
                elif re_CC.match(line):
                    CC = re_CC.findall(line)[0]
                elif re_BCC.match(line):
                    Bcc = re_BCC.findall(line)[0]
                elif re_Subject.match(line):
                    Subject = re_Subject.findall(line)[0]
                elif re_Attachments.match(line):
                    Attachments = re_Attachments.findall(line)[0]
                elif re_Importance.match(line):
                    Importance = re_Importance.findall(line)[0]
                    
                elif re_isThread.match(line):
                    isThread = True
                elif re_isAutoMessage.match(line):
                    isAutoMessage = True
                elif re_isDisplacement.match(line):
                    isDisplacement = True
                elif re_hasAllCapLine.match(line):
                    hasAllCapLine = True
                elif re_hasBadDate.match(line):
                    hasBadDate = True
                    
                elif re_Body.match(line):
                    foundBody = True
#                     line = re.sub('[^A-Za-z0-9\s\(\)\/\\\&\?\!\,\.\-\@\<\>\:\+\%\;]', '', line)
#                     list_body.append(line) # the Body: line is not append
    
    dictionary['From'] = From
    dictionary['Sent'] = Sent
    dictionary['To'] = To
    dictionary['CC'] = CC
    dictionary['Bcc'] = Bcc
    dictionary['Subject'] = Subject
    dictionary['Attachments'] = Attachments
    dictionary['Importance'] = Importance
    
    
    dictionary['isThread'] = isThread
    dictionary['isAutoMessage'] = isAutoMessage
    dictionary['isDisplacement'] = isDisplacement
    dictionary['hasAllCapLine'] = hasAllCapLine
    dictionary['hasBadDate'] = hasBadDate
    
    dictionary['Body'] = list_body

    return dictionary

In [16]:
# inputpath = './split_txt/' 
# inputpath = './redacted_split_txt/' 
# inputpath = './unredacted_split_txt/' 
inputpath = './test_output//' 
# inputpath = './old_split_txt/' 
l = []
for name in tqdm_notebook(os.listdir(inputpath)):
    if name != '.DS_Store':
        l.append(get_headers(inputpath+'/'+name))
len(l)

/Users/joey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


78869

In [17]:
csv_columns = ['filenm','From','Sent','To','CC','Bcc','Subject','Attachments','Importance','isThread','isAutoMessage','isDisplacement','hasAllCapLine','hasBadDate','Body']
# with open('./output.csv', 'w',encoding='utf8') as csvfile:
# with open('./redacted_output.csv', 'w',encoding='utf8') as csvfile:
# with open('./unredacted_output.csv', 'w',encoding='utf8') as csvfile:
with open('./output_0211.csv', 'w',encoding='utf8') as csvfile:
# with open('./old_output.csv', 'w',encoding='utf8') as csvfile:

    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for data in l:
        writer.writerow(data)

In [18]:
df = pd.read_csv('output_0211.csv')
df

,From,Sent,To,CC,Bcc,Subject,Attachments,Importance,isThread,isAutoMessage,isDisplacement,hasAllCapLine,hasBadDate,Body
0,Davisson Dunlap III [mailto:davissoniii@dunlap...,"Wednesday, February 11, 2015 10:07 AM","Beaudoin, Mark",Kim Rivers,[],RE: Existing Sidewalk Cafe Lease,[],[],False,False,False,False,False,"['Body:', 'Thank you, Mark.', 'Im attaching th..."
1,Ruth Draper,"Thursday, September 24, 2015 at 10:13:40 AM Ea...",NARAYAN.PERSAUD@comcast.net,"William HaZield, Byron Dobson, sfoster@tallah...",[],thanks for your thoughts - FAMU needs more ac_on,[],[],False,False,False,False,False,"['Body:', 'proctorb@leoncountyFL.gov, andrew.g..."
2,"Moore, Doug","Monday, July 09, 2012 2:18 PM",'KAVALLONE@JOHNRYANCO.COM'; 'KBLAIR1046@AOL.CO...,"Jumonville, Karen; Baker, Dale",[],"City Of Tallahassee, Building Inspection Staff...",[],[],False,False,False,False,False,"['Body:', 'Sent On behalf of Karen Jumonville,..."
3,Paul LaMaster <PAULLAMASTER250@comcast.net,"July 26, 2015 8:45:10 AM EDT","""lefers@tallahassee.com"" <lefers@tallahassee.com","andrew.gillum@talgov.com, alan.williams@myﬂori...",[],Unimpressed with Chicago,[],[],False,False,False,False,False,"['Body:', 'I was totally unimpressed with this..."
4,Martin Merzer <marty@martinmerzer.com,"Thursday, April 11, 2013 11:41 AM",Allie Fleming,[],[],Downtown Improvement Authority Recommendation ...,[],[],False,False,False,False,False,"['Body:', '', 'isDisplacement: True']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78864,"Brown, Jessica","Friday, January 13, 2017 6:39 PM","Edwards, Ashley","Fendley, Daniel",[],Re: Edison Outdoor Lawn Photo,[],[],False,False,False,False,False,"['Body:', 'You are SOOO good! Thank you!!', 'S..."
78865,Davisson Dunlap III <davissoniii@dunlapship ma...,"November 23, 2015 at 8:03:37 AM HST",Kim Rivers <kim@inkbridge.com,[],[],Fwd:,[],[],False,False,False,False,False,"['Body:', 'Doubletree - PorteCochere Easement ..."
78866,"Fleming, Allie","Tuesday, April 02, 2013 3:36 PM","Capps, Danny; Faris, Alison",[],[],RE: FYI ... Subscriber option for SM newslette...,[],[],False,False,False,False,False,"['Body:', 'From what I can tell it is still do..."
78867,"Kring, Edward","Wednesday, February 22, 2017 at 3:59:54 PM Eas...",Paige Carter-Smith,[],[],Trolley mee-ng,[],[],False,False,False,False,False,"['Body:', '', 'isDisplacement: True']"
